In [2]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

# Set display spot
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [3]:
# Define data paths
spot_path = Path('../raw/spot/feather/binance/BTCUSDT')

print("=== DATA OVERVIEW ===")

print("\n--- SPOT ---")

if not spot_path.exists():
    print("No spot data found")
else:
    files = sorted(list(spot_path.glob('*.feather')))
    if not files:
        print("No spot files found")
    else:
        print(f"Files: {len(files)} ({files[0].stem} to {files[-1].stem})")
        
        # Load a sample file from the middle
        sample_file = files[len(files)//2]
        df = pd.read_feather(sample_file)
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='us', utc=True)

        print(df.head(10))
        
        print(f"Sample file: {sample_file.name}")
        print(f"   Rows: {len(df):,}")
        print(f"   Columns: {list(df.columns)}")
        
        if 'timestamp' in df.columns:
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='us', utc=True)
            print(f"   Time range: {df['datetime'].min()} to {df['datetime'].max()}")
        
        if 'price' in df.columns:
            print(f"   Price range: ${df['price'].min():.4f} - ${df['price'].max():.4f}")
        
        if 'instrument' in df.columns:
            print(f"   Instruments: {df['instrument'].nunique()} unique")
            print(f"   Top 3: {df['instrument'].value_counts().head(3).index.tolist()}")

=== DATA OVERVIEW ===

--- SPOT ---
Files: 181 (2024-01-02 to 2024-06-30)
          timestamp          id     price  quantity  is_buyer_maker  \
0  1711929600000000  3523985836  71280.00   0.00141            True   
1  1711929600001000  3523985837  71280.00   0.00075            True   
2  1711929600003000  3523985838  71280.00   0.00045            True   
3  1711929600005000  3523985839  71280.00   0.00078            True   
4  1711929600006000  3523985840  71280.01   0.00132           False   
5  1711929600010000  3523985841  71280.00   0.00044            True   
6  1711929600011000  3523985842  71280.00   0.00117            True   
7  1711929600011000  3523985843  71280.00   0.00020            True   
8  1711929600011000  3523985844  71280.01   0.00028           False   
9  1711929600011000  3523985845  71280.01   0.00066           False   

                          datetime  
0        2024-04-01 00:00:00+00:00  
1 2024-04-01 00:00:00.001000+00:00  
2 2024-04-01 00:00:00.003000+00:0

In [8]:
df['datetime'] = pd.to_datetime(df['datetime'], utc=True)

# Group into 1-second bins
spot_1s = (
    df.set_index('datetime')
      .resample('1s')
      .agg({
          'price': 'mean',       # average price in that second
          'quantity': 'sum',     # total volume in that second
      })
      .dropna()  # remove seconds with no trades
)

spot_1s.rename(columns={'price': 'mid_price ($)', 'quantity': 'volume'}, inplace=True)

print(spot_1s.head())
print(spot_1s.tail())
print(len(spot_1s))


                           mid_price ($)   volume
datetime                                         
2024-04-01 00:00:00+00:00   71280.003469  0.58399
2024-04-01 00:00:01+00:00   71284.796239  8.49194
2024-04-01 00:00:02+00:00   71273.543356  6.15201
2024-04-01 00:00:03+00:00   71258.078077  5.44813
2024-04-01 00:00:04+00:00   71250.006818  0.44479
                           mid_price ($)   volume
datetime                                         
2024-04-01 23:59:55+00:00   69649.805000  0.00189
2024-04-01 23:59:56+00:00   69649.806000  0.01421
2024-04-01 23:59:57+00:00   69649.810000  0.00331
2024-04-01 23:59:58+00:00   69649.805000  0.00094
2024-04-01 23:59:59+00:00   69649.805333  0.21433
84552


In [ ]:
spot_path = Path("../raw/spot/feather/binance/BTCUSDT")
output_file = Path("../parsed/spot/btc_spot_1s.parquet")


spot_files = sorted(spot_path.glob("*.feather"))
print(f"Found {len(spot_files)} files")

spot_1s_list = []

for f in spot_files:
    df = pd.read_feather(f)
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='us', utc=True)
    
    df_1s = (
        df.set_index('datetime')
          .resample('1s')
          .agg({
              'price': ['first', 'max', 'min', 'last'],
              'quantity': 'sum'
          })
          .dropna()
    )
    df_1s.columns = ['open', 'high', 'low', 'close', 'volume']
    spot_1s_list.append(df_1s)
    
    print(f"Processed {f.name} → {len(df_1s)} seconds")

# Combine all daily 1s bars
spot_1s = pd.concat(spot_1s_list).sort_index()

# Fill missing seconds
spot_1s = spot_1s.asfreq('1S').ffill()

spot_1s.to_parquet(output_file)
print(f"Saved {output_file} with {len(spot_1s):,} rows")



Found 181 files


/var/folders/nn/80hngvbx12l0bp84pd944rqw0000gn/T/ipykernel_26251/1128474090.py:15: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .resample('1S')


Processed 2024-01-02.feather → 85444 seconds


/var/folders/nn/80hngvbx12l0bp84pd944rqw0000gn/T/ipykernel_26251/1128474090.py:15: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .resample('1S')


Processed 2024-01-03.feather → 84601 seconds


...

Processed 2024-06-28.feather → 79653 seconds
Processed 2024-06-29.feather → 76915 seconds


/var/folders/nn/80hngvbx12l0bp84pd944rqw0000gn/T/ipykernel_26251/1128474090.py:15: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .resample('1S')


Processed 2024-06-30.feather → 76365 seconds


/var/folders/nn/80hngvbx12l0bp84pd944rqw0000gn/T/ipykernel_26251/1128474090.py:31: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  spot_1s = spot_1s.asfreq('1S').ffill()


Saved btc_spot_1s.parquet with 15,638,400 rows


In [13]:
file_path = "../parsed/spot/btc_spot_1s.parquet"

df = pd.read_parquet(file_path)

# Overview
print("=== File Overview ===")
print(f"Rows: {len(df):,}")
print(f"Columns: {list(df.columns)}")
print(f"Time range: {df.index.min()} to {df.index.max()}")

# Peek at first 5 rows
print("\n--- Head ---")
print(df.head())

# Peek at last 5 rows
print("\n--- Tail ---")
print(df.tail())

# Summary stats
print("\n--- Summary statistics ---")
print(df.describe())

=== File Overview ===
Rows: 15,638,400
Columns: ['open', 'high', 'low', 'close', 'volume']
Time range: 2024-01-02 00:00:00+00:00 to 2024-06-30 23:59:59+00:00

--- Head ---
                               open      high       low     close   volume
datetime                                                                  
2024-01-02 00:00:00+00:00  44179.55  44183.68  44179.54  44183.67  3.22085
2024-01-02 00:00:01+00:00  44183.67  44183.67  44174.30  44174.30  9.47101
2024-01-02 00:00:02+00:00  44174.30  44174.30  44169.61  44173.67  9.26707
2024-01-02 00:00:03+00:00  44173.68  44181.55  44173.68  44181.54  4.63683
2024-01-02 00:00:04+00:00  44181.54  44181.55  44181.54  44181.55  0.18476

--- Tail ---
                               open      high      low     close   volume
datetime                                                                 
2024-06-30 23:59:55+00:00  62772.00  62772.00  62772.0  62772.00  0.00167
2024-06-30 23:59:56+00:00  62772.01  62772.01  62772.0  62772.00  0

In [18]:
# === CONFIG ===
options_path = Path("../raw/options/feather/deribit/BTC")  # folder with raw option files
output_file = Path("../btc_options_2024-01-02_2024-01-09.parquet")
output_file.parent.mkdir(parents=True, exist_ok=True)

# Date filter (UTC)
start_date = pd.Timestamp("2024-01-02", tz="UTC")
end_date   = pd.Timestamp("2024-01-09", tz="UTC")

option_files = sorted(options_path.glob("*.feather"))
print(f"Found {len(option_files)} files")

dfs = []
for f in option_files:
    df = pd.read_feather(f)
    
    # Convert timestamp
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='us', utc=True)
    
    # Filter by date range
    df = df[(df['datetime'] >= start_date) & (df['datetime'] < end_date)]
    
    if not df.empty:
        dfs.append(df)
        print(f"Loaded {f.name} → {len(df):,} rows after filtering")

# Combine and sort
options = pd.concat(dfs, ignore_index=True).sort_values("datetime")

# Save filtered file
options.to_parquet(output_file)
print(f"Saved {output_file} with {len(options):,} rows")
print(f"Time range: {options['datetime'].min()} → {options['datetime'].max()}")


Found 182 files
Loaded 2024-01-02.feather → 17,478 rows after filtering
Loaded 2024-01-03.feather → 20,820 rows after filtering
Loaded 2024-01-04.feather → 15,180 rows after filtering
Loaded 2024-01-05.feather → 16,979 rows after filtering
Loaded 2024-01-06.feather → 9,198 rows after filtering
Loaded 2024-01-07.feather → 8,435 rows after filtering
Loaded 2024-01-08.feather → 24,544 rows after filtering
Saved ../btc_options_2024-01-02_2024-01-09.parquet with 112,634 rows
Time range: 2024-01-02 00:00:00.593000+00:00 → 2024-01-08 23:59:48.600000+00:00


In [34]:
spot_file = Path("../btc_spot_1s.parquet")
opt_file  = Path("../btc_options_2024-01-02_2024-01-09.parquet")

# Load files
spot = pd.read_parquet(spot_file)
options = pd.read_parquet(opt_file)

print("=== SPOT DATA ===")
print(f"Rows: {len(spot):,} | Columns: {list(spot.columns)}")
if 'datetime' in spot.columns:
    print(f"Time range: {spot['datetime'].min()} → {spot['datetime'].max()}")
else:
    print(f"Index range: {spot.index.min()} → {spot.index.max()}")
print("\nHead:\n", spot.head(), "\nTail:\n", spot.tail())

# Always show all columns without truncation
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

print("\n=== OPTIONS DATA ===")
print(f"Rows: {len(options):,} | Cols: {options.shape[1]}")
print(f"Columns: {list(options.columns)}")

if 'datetime' in options.columns:
    print(f"Time range: {options['datetime'].min()} → {options['datetime'].max()}")
else:
    print(f"Index range: {options.index.min()} → {options.index.max()}")



=== SPOT DATA ===
Rows: 15,638,400 | Columns: ['open', 'high', 'low', 'close', 'volume']
Index range: 2024-01-02 00:00:00+00:00 → 2024-06-30 23:59:59+00:00

Head:
                                open      high       low     close   volume
datetime                                                                  
2024-01-02 00:00:00+00:00  44179.55  44183.68  44179.54  44183.67  3.22085
2024-01-02 00:00:01+00:00  44183.67  44183.67  44174.30  44174.30  9.47101
2024-01-02 00:00:02+00:00  44174.30  44174.30  44169.61  44173.67  9.26707
2024-01-02 00:00:03+00:00  44173.68  44181.55  44173.68  44181.54  4.63683
2024-01-02 00:00:04+00:00  44181.54  44181.55  44181.54  44181.55  0.18476 
Tail:
                                open      high      low     close   volume
datetime                                                                 
2024-06-30 23:59:55+00:00  62772.00  62772.00  62772.0  62772.00  0.00167
2024-06-30 23:59:56+00:00  62772.01  62772.01  62772.0  62772.00  0.06558
2024-06